In [28]:
shape_cat=""
vol="uber"
z_range=""

In [29]:
shape_cat="mcal"
vol="uber"
z_range="all_z"

In [30]:
#This header file contains all the common packages
%matplotlib inline
%run /home/cz136/project/sa/lib/src/header.py $shape_cat $vol $z_range 
%run -i /home/cz136/project/sa/lib/src/func.py  

In [40]:
z_diff=0.2
z_diff_string="02"

In [41]:
foreground=shapes[(shapes[('All','Z')]-shapes[('All','ZRED2')])>=z_diff]
background=shapes[(shapes[('All','Z')]-shapes[('All','ZRED2')])<=-z_diff]

In [42]:
background.to_pickle('/home/cz136/project/sa/data/{}/{}/background{}.pkl'.format(shape_cat,vol,z_diff_string))

In [43]:
foreground.to_pickle('/home/cz136/project/sa/data/{}/{}/foreground{}.pkl'.format(shape_cat,vol,z_diff_string))